# Rotational-vibrational solutions for H2S molecule

In [1]:
import jax
import numpy as np
from jax import config
from jax import numpy as jnp
from numpy.polynomial.hermite import hermgauss
from scipy import optimize

from rovib.h2s_potential import potential
from rovib.keo import Molecule, batch_Gmat, batch_pseudo, com
from rovib.primbas import hermite
from rovib.symtop import rotme_cor, rotme_ovlp, rotme_rot
from rovib.vib_xy2 import vibrations_xy2
from rovib import c2v

config.update("jax_enable_x64", True)

Define atomic masses and internal coordinates for describing vibrations. Here, we use valence-bond coordinates, $r_1\equiv \text{S--H}_1$, $r_2\equiv \text{S--H}_2$, and $\alpha = \angle\text{H}_1\text{SH}_2$.

The function `valence_bond_coordinates` is required to build the kinetic energy operator.

In [2]:
NCOO = 3 # number of vibrational coordinates
MASS_S = 31.97207070
MASS_H = 1.00782505

@com
def valence_bond_coordinates(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Molecule.masses = np.array([MASS_S, MASS_H, MASS_H])
Molecule.internal_to_cartesian = valence_bond_coordinates

We will employ Hermite functions $H_n(x)e^{-x^2/2}$ as vibrational basis functions and Gauss-Hermite quadratures for computing matrix elements.
The Hermite functions are defined on the coordinate range $x\in(-\infty,\infty)$. In order to map $x$ into our vibrational valence bond coordinates $r_1\in(0,\infty)$, $r_2\in(0,\infty)$, and $\alpha=(0,\pi)$, we use linear function $r=ax+b$

In [3]:
vmin = optimize.minimize(potential, [1.0, 1.0, np.pi / 2])
r0 = vmin.x
v0 = vmin.fun
print("min of the potential:", r0, v0)

freq = np.diag(jax.hessian(potential)(r0))  # NOTE multiply by 2?
mu = np.diag(batch_Gmat(jnp.array([r0]))[0, :NCOO, :NCOO])
lin_a = np.sqrt(np.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a' and 'b':", lin_a, lin_b)

x_to_r_map = lambda x: lin_a * x + lin_b

min of the potential: [1.3358387  1.3358387  1.61042427] -0.0007846164037204629
x->r linear mapping parameters 'a' and 'b': [0.11245619 0.11245619 0.17845517] [1.3358387  1.3358387  1.61042427]


In [4]:
# list of 1D primitive basis functions for each vibrational coordinate
list_psi = (hermite, hermite, hermite)

# number of primitive basis functions for each vibrational coordinate
nmax = 20

# number of quadrature points for each vibrational coordinate
npoints = 60

vec = []

for icoo in range(NCOO):
    print(f"\nSolve for coordinate #{icoo}")

    # quanta, set quanta to zero for all coordinates except `icoo`
    list_q = [np.arange(nmax) if i == icoo else [0] for i in range(NCOO)]

    # quadratures, reduce number of points for all coordinates except `icoo`
    n = [npoints if i == icoo else 10 for i in range(NCOO)]
    n1, n2, n3 = n
    x1, w1 = hermgauss(n1)
    x2, w2 = hermgauss(n2)
    x3, w3 = hermgauss(n3)
    w1 /= np.exp(-(x1**2))
    w2 /= np.exp(-(x2**2))
    w3 /= np.exp(-(x3**2))
    list_x = (x1, x2, x3)
    list_w = (w1, w2, w3)

    # solve for basis functions for `icoo` vibrational coordinate
    e, v, *_ = vibrations_xy2(
        list_psi,
        list_x,
        list_w,
        list_q,
        select_points=lambda x, w: True,
        select_quanta=lambda q: np.sum(q * np.array([1, 1, 1])) <= nmax,
        x_to_r_map=x_to_r_map,
        gmat=lambda x: batch_Gmat(x),
        pseudo=batch_pseudo,
        potential=potential,
    )

    # keep eigenvectors
    vec.append(v)

    print(f"energies:")
    print(e[0], e - e[0])


Solve for coordinate #0
energies:
3337.3945251332743 [     0.           2622.78234885   5150.20518524   7582.4678558
   9919.70554205  12164.50975541  14343.30102081  16563.82374237
  18993.88593035  21727.63458668  24794.39790399  28188.39606251
  31915.83174881  36157.40998912  40470.55415716  45509.19492507
  55777.84304183  78535.64469475 130761.97020122 292130.27272365]

Solve for coordinate #1
energies:
3337.3945251420123 [     0.           2622.78234885   5150.20518524   7582.4678558
   9919.70554205  12164.5097554   14343.30102081  16563.82374236
  18993.88593031  21727.63458661  24794.39790389  28188.39606238
  31915.83174865  36157.40998892  40470.55415693  45509.19492476
  55777.84304143  78535.64469421 130761.97020033 292130.27272174]

Solve for coordinate #2
energies:
3381.0291550488237 [    0.          1214.01682721  2422.18421732  3624.08614385
  4819.27800458  6007.28844615  7187.64292047  8359.92027977
  9523.84423656 10679.39786578 11826.92835376 12967.2493325
 14101

Transform primitive basis to the basis of eigenfunctions of three solutions and solve 3D problem in full basis.

In [5]:
# primitive to contracted basis
list_psi = [
    lambda x, n: jnp.dot(hermite(x, n), vec[0]),
    lambda x, n: jnp.dot(hermite(x, n), vec[1]),
    lambda x, n: jnp.dot(hermite(x, n), vec[2]),
]

In [6]:
gr.shape

NameError: name 'gr' is not defined